In [ ]:
cd /content/drive/My Drive/GSoC2020/audio_visual_emotion/GSoC2020/

/content/drive/My Drive/GSoC2020/audio_visual_emotion/GSoC2020


In [ ]:
import numpy as np
import os
import sys

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers.convolutional import Conv2D
from keras.layers import LSTM, Input, Flatten, Embedding, Convolution1D,Dropout, Concatenate
from keras.optimizers import SGD, Adam, RMSprop

from scipy import signal
from sklearn.preprocessing import label_binarize

import pickle
from features import *
from helper import *

## Loading data 

In [ ]:
code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
#data_path = code_path + "/../data/sessions/"
data_path = "../../Data/IEMOCAP/"
framerate = 16000

In [ ]:
# Load IEMOCAP data
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [ ]:
# Here we only use the facial features since they are predominately available in a video
# Besides, using hand and head alone has an accuracy close to 0.25

mocap_rot = []

for ses_mod in data2:
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))  
    x_rot[np.isnan(x_rot)]=0
    mocap_rot.append( x_rot )
    
mocap_rot = np.array(mocap_rot)
mocap_rot.shape

(950, 200, 165)

In [ ]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(950, 4)

## Training Visual-only models

In [ ]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(256, return_sequences=False, input_shape=(200, 6)))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

    return model

model = lstm_model()
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               269312    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 516       
_________________________________________________________________
activation_4 (Activation)    (None, 4)                 0         
Total params: 302,724
Trainable params: 302,724
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist = model.fit(mocap_head, Y, 
                 batch_size=100, nb_epoch=25, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 760 samples, validate on 190 samples
Epoch 1/25
760/760 [==============================] - 3s 4ms/step - loss: 1.3217 - accuracy: 0.4303 - val_loss: 1.3518 - val_accuracy: 0.3316
Epoch 2/25
760/760 [==============================] - 3s 3ms/step - loss: 1.2616 - accuracy: 0.4842 - val_loss: 1.3959 - val_accuracy: 0.2684
Epoch 3/25
760/760 [==============================] - 3s 4ms/step - loss: 1.2194 - accuracy: 0.5237 - val_loss: 1.3777 - val_accuracy: 0.3737
Epoch 4/25
760/760 [==============================] - 3s 4ms/step - loss: 1.1873 - accuracy: 0.5211 - val_loss: 1.3920 - val_accuracy: 0.3895
Epoch 5/25
760/760 [==============================] - 3s 3ms/step - loss: 1.1633 - accuracy: 0.5276 - val_loss: 1.4102 - val_accuracy: 0.3842
Epoch 6/25
760/760 [==============================] - 3s 3ms/step - loss: 1.1433 - accuracy: 0.5368 - val_loss: 1.4148 - val_accuracy: 0.3947
Epoch 7/25
760/760 [==============================] - 3s 3ms/step - loss: 1.1250 - accuracy: 0.5461 - 

In [ ]:

def simple_conv(optimizer='Adam'):
    model = Sequential()
    model.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 165, 1)))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Activation('relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

model = simple_conv()
model.summary()

mocap_rot_re = mocap_rot.reshape(-1,200,165,1)

hist = model.fit(mocap_rot_re, Y, 
                 batch_size=100, nb_epoch=50, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, strides=(2, 2), input_shape=(200, 165,..., padding="same")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 83, 32)       320       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 83, 32)       0         
_________________________________________________________________
activation_5 (Activation)    (None, 100, 83, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 42, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 42, 64)        0         
_________________________________________________________________
activation_6 (Activation)    (None, 50, 42, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 21, 64)       

#### Video2landmarks

In [ ]:
!pip install face-alignment

In [32]:
!python video2landmarks.py --video_path='test/obama.mp4' --save_path='test/obama/' --device='cuda'

Namespace(device='cuda', save_path='test/obama/', video_path='test/obama.mp4')
Extracting landmarks:
100% 253/253 [00:53<00:00,  4.74it/s]
Landmarks saved at test/obama/landmarks.pickle


In [ ]:
import pickle
with open('test/obama/landmarks.pickle', 'rb') as handle:
    lm = pickle.load(handle)

In [33]:
len(lm)

253

In [34]:
!git 

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   audio_visual.ipynb
	modified:   video2landmarks.py
	modified:   visual_only.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	test/

no changes added to commit (use "git add" and/or "git commit -a")
